In [ ]:
# deps for model
import sys
import random
import torch
import torchvision
# deps for data storing
from datetime import datetime
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
random.seed(0)

## Zmienne potrzebne do zebrania danych

In [1]:
columns = ["id", "original_label", "classifier", "features"]
collected_data = []
def get_features(model, data):
    # print(data.shape)
    # print("---")
    # print(model._modules["avgpool"](data).shape)
    for name, module in model._modules.items():
        data = module(data)
        # print(name, data.shape)
        if name == "avgpool":
          #tu są cechy które chcę wyciągnąć z sieci
          data = data.view(-1, 2048)
          return data
dataset_name = "cifar-10"

In [ ]:
!gdown 1YSs2XjzvAM_0ZbttiznTDo7GUdCX1erK #ResNet152_CIFAR10
!gdown 1ceCGuo_3U849gzDbEPn0GDNuM9No73gI #test_cifar10

Downloading...
From: https://drive.google.com/uc?id=1YSs2XjzvAM_0ZbttiznTDo7GUdCX1erK
To: /content/ResNet152_CIFAR10.ckpt
100% 466M/466M [00:10<00:00, 44.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ceCGuo_3U849gzDbEPn0GDNuM9No73gI
To: /content/test_cifar10.pickle
100% 83.0M/83.0M [00:02<00:00, 40.0MB/s]


In [ ]:
# pobranie modelu restnet
model = torchvision.models.resnet152(num_classes=10)

model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()
# tworzenie modelu
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
# wczytywanie
state_dict = torch.load("./ResNet152_CIFAR10.ckpt", map_location='cuda:0')["state_dict"]
for key in list(state_dict.keys()):
    state_dict[key.replace('model.', '')] = state_dict.pop(key)

model.load_state_dict(state_dict)
model.cuda()
model.eval()
print("Model loaded")


Model loaded


In [ ]:
# wczytywanie testsetu cifar
test_set = torchvision.datasets.CIFAR10(root="./", train=False, download=True, transform=preprocess)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

# (32, 3, 32, 32) Wymiarowość pojedynczego batch'a - 32 obrazki 3 kolory RGB, wymiary 32x32
total, correct = 0, 0
index = 0
# Brak gradientowosci
with torch.no_grad():
  for i_batch, (data, targets) in enumerate(test_loader):
      data, targets = data.cuda(), targets.cuda()
      # logits- czytaj przestrzeń wartości dla konkretnych klass
      logits = model(data)
      # pobieramy wagi z wybranej warstwy i przekształcamy do naszych potrzeb
      features = get_features(model, data)
      for i in range(logits.shape[0]):
        collected_data.append([index, targets[i].item(), logits[i].detach().cpu().numpy(), features[i].detach().cpu().numpy()])
        index += 1
      _, predicted = torch.max(logits, 1)
      # print()
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

print("Acc", correct/total)
print("Total", total)



100%|██████████| 170498071/170498071 [00:01<00:00, 100318628.48it/s]


Extracting ./cifar-10-python.tar.gz to ./
conv1 torch.Size([32, 64, 32, 32])
bn1 torch.Size([32, 64, 32, 32])
relu torch.Size([32, 64, 32, 32])
maxpool torch.Size([32, 64, 32, 32])
layer1 torch.Size([32, 256, 32, 32])
layer2 torch.Size([32, 512, 16, 16])
layer3 torch.Size([32, 1024, 8, 8])
layer4 torch.Size([32, 2048, 4, 4])
avgpool torch.Size([32, 2048, 1, 1])
conv1 torch.Size([32, 64, 32, 32])
bn1 torch.Size([32, 64, 32, 32])
relu torch.Size([32, 64, 32, 32])
maxpool torch.Size([32, 64, 32, 32])
layer1 torch.Size([32, 256, 32, 32])
layer2 torch.Size([32, 512, 16, 16])
layer3 torch.Size([32, 1024, 8, 8])
layer4 torch.Size([32, 2048, 4, 4])
avgpool torch.Size([32, 2048, 1, 1])
conv1 torch.Size([32, 64, 32, 32])
bn1 torch.Size([32, 64, 32, 32])
relu torch.Size([32, 64, 32, 32])
maxpool torch.Size([32, 64, 32, 32])
layer1 torch.Size([32, 256, 32, 32])
layer2 torch.Size([32, 512, 16, 16])
layer3 torch.Size([32, 1024, 8, 8])
layer4 torch.Size([32, 2048, 4, 4])
avgpool torch.Size([32, 2048,

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# print(collected_data)

def save_to_pickle_file(data: list, columns: list, filename: str ):
  """ Save collected data about model performance to pickle file


  """
  df = pd.DataFrame(data, columns = columns)
  print(df.head())
  df.to_pickle(f"/content/drive/MyDrive/master_degree/{filename}.pickle")

save_to_pickle_file(collected_data, columns, "cifar_10")



   id  original_label                                         classifier  \
0   0               3  [-1.9221079, -0.05551105, -1.7079026, 12.52216...   
1   1               8  [0.2173618, 0.605438, -1.8011787, -1.4414213, ...   
2   2               8  [-0.048723668, 5.0532928, -1.9844537, -2.31005...   
3   3               0  [8.810869, 0.05846511, -1.66308, 0.9205174, -3...   
4   4               6  [-2.2548902, -0.9673878, 0.12587366, -0.154412...   

                                            features  
0  [0.39855704, 0.09600668, 0.006940333, 0.000680...  
1  [0.007957501, 0.08981927, 0.0, 0.0, 0.01822564...  
2  [0.0011582605, 0.030336332, 0.0, 0.0, 0.120753...  
3  [0.21992034, 0.043670937, 0.0, 0.1730167, 0.13...  
4  [0.0017583021, 0.002734655, 0.0, 0.0, 0.414119...  
